In [21]:
"""
To be able to do transfer learning a base model needs to be trained

1. Read a subset of FairFace data set. TODO: Read entire FairFace data set
2. Build base model
    2.1 Choose some model. TODO: Be specific about model chosen
    2.2 Random weights
    2.3 Input size as appropriate
3. Train base model
"""

'\nTo be able to do transfer learning a base model needs to be trained\n\n1. Read a subset of FairFace data set. TODO: Read entire FairFace data set\n2. Build base model\n    2.1 Choose some model. TODO: Be specific about model chosen\n    2.2 Random weights\n    2.3 Input size as appropriate\n3. Train base model\n'

In [22]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from fairness import fairnessMetrics as fm

In [7]:
!pip install keras-efficientnet

In [27]:
# TODO: Replace with Christophs update code for loading data sets. 
def readImages(folderName, image_size, batch_size):
    train_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
        labels='inferred'
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
        labels='inferred'
    )
    return (train_ds, val_ds)

In [41]:
# Build base model
# TODO: Choose appropriate model
base_model = keras.applications.Xception(
    weights=None, 
    input_shape=(299, 299, 3),
    include_top=True,
    classes = 2,
)

In [42]:
# 
metrics_list = ["accuracy",
                tf.keras.metrics.TruePositives(), 
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.FalsePositives(), 
                tf.keras.metrics.FalseNegatives(), 
                fm.TruePositiveRate(),
                fm.TrueNegativeRate(),
                fm.FalsePositiveRate(),
                fm.FalseNegativeRate(),
                fm.PositivePredictedValue(),
                fm.FalseDiscoveryRate(),
                fm.NegativePredictedValue(),
                fm.FalseOmissionRate(),
                fm.BinaryDemographicParityDiff(),
                fm.DemographicParity(),
                fm.BinaryEqualizedOddsDiff(),
                fm.BinaryProportionalParityDiff(),
                fm.ProportionalParity(),
                fm.BinaryPredictiveRateParityDiff(),
                fm.PredictiveRateParity(),
                fm.BinaryAccuracyParityDiff(),
                fm.AccuracyParity(),
                fm.BinaryFalseNegativeRateParityDiff(),
                fm.BinaryFalsePositiveRateParityDiff(),
                fm.BinaryNegativePredictiveRateParityDiff(),
                fm.NegativePredictiveRateParity(),
                fm.BinarySpecificityParityDiff()]

optimizer = keras.optimizers.Adam(learning_rate=1e-3) # TODO: Choose appropriate optimizer and learning_rate

base_model.compile(optimizer=optimizer, 
                   loss="binary_crossentropy", # TODO: look into this
                   metrics=metrics_list
)

In [ ]:
"""
Could be implemented?: Batch the data and use caching & prefetching to optimize loading speed
"""

In [ ]:
"""
DATA AUGMENTATION COULD BE ADDED HERE
"""

In [43]:
image_size = (299, 299) # This step should be standardizing, should be able to be changed to what is approptiate
batch_size = 32
data = readImages("FairFace_subset", image_size, batch_size)

Found 200 files belonging to 2 classes.
Using 160 files for training.
Found 200 files belonging to 2 classes.
Using 40 files for validation.


In [44]:
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [45]:
epochs = 20
base_model.fit(data[0], epochs=epochs, validation_data=data[1])

Epoch 1/20


ValueError: in user code:

    /home/erbuntno/.local/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/erbuntno/.local/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/erbuntno/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/erbuntno/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/erbuntno/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/erbuntno/.local/lib/python3.8/site-packages/keras/engine/training.py:835 run_step  **
        outpu

In [ ]:
base_model.save('/tmp/model')